# STRIDE-CH

Sub-Transition Region Identification of Ensemble Coronal Holes

1. Specify file system settings in the settings.py script
2. Load, download, and rename data in the **Data Preparation** section.
3. Confirm that the desired data has been loaded into memory in the **Available Data** section.
4. Produce desired data products, including ensemble CH segmentation maps in the **Data Products** section.
5. Produce desired plots of data products in the **Data Product Plots** section.

In [ ]:
import os
# See README Python Dependency Installation
os.environ['IMAGEIO_FFMPEG_EXE'] = '/opt/homebrew/bin/ffmpeg'

import pickle
import sunpy.map
import numpy as np
from sunpy.net import Fido, attrs as a
from datetime import datetime, timedelta

import astropy.units as u
from astropy.coordinates import SkyCoord
from matplotlib import colormaps, pyplot as plt

import prepare_data
import detect
import plot_detection
from settings import *

%matplotlib inline

# Data Preparation

To prepare previously downloaded data to be accessed by this notebook, simply run the **Load Data** subsection.

To download and prepare new data:
1. Visit the links provided in the **Data** section of the README.
2. Manually download the desired He I images and corresponding magnetograms to the subdirectories within the `STRIDE-CH` directory. The default subdirectories are provided below, but may be customized in the settings.py script.
   - He I Images: `STRIDE-CH/assets/He`
   - Magnetograms: `STRIDE-CH/assets/Mag`
3. Execute the **Rename Data** subsection below to rename these files.
4. Execute the **Load Data** subsection.
5. Use the **EUV Download** subsection below to download corresponding EUV 19.3 nm images.
6. Re-execute the **Rename Data** and **Load Data** subsections.

## Load Data

Load lists of available datetimes of observations in the file system into memory.

For the usage of ACWE maps, the `DATE_RANGE` in settings.py must encompass all available ACWE map files.

In [ ]:
# Extract He I observation datetimes from FITS files
HE_DATE_LIST = prepare_data.get_fits_date_list(
    DATE_RANGE, HE_DIR
)

# Extract magnetogram datetimes from 6302l FITS files
MAG_DATE_LIST = prepare_data.get_fits_date_list(
    DATE_RANGE, MAG_DIR
)

# Extract EUV datetimes from FITS files
EUV_DATE_LIST = prepare_data.get_fits_date_list(
    DATE_RANGE, EUV_DIR
)

# Extract ACWE datetimes from FITS files
ACWE_DATE_LIST = prepare_data.get_acwe_date_list(DATE_RANGE)

if HE_DATE_LIST or EUV_DATE_LIST:
    date_strs = [HE_DATE_LIST[0], HE_DATE_LIST[-1]]
    file_date_str = f'{date_strs[0]}_{date_strs[-1]}'

    num_maps = len(HE_DATE_LIST)
    datetimes = [datetime.strptime(date_str, DICT_DATE_STR_FORMAT)
                for date_str in date_strs]
    title_date_strs = [datetime.strftime(d, '%m/%d/%Y') for d in datetimes]
    DATE_RANGE_SUPTITLE = (f'{num_maps} Maps Evaluated from '
                        + f'{title_date_strs[0]} to {title_date_strs[-1]}')
else:
    print('No data is available for the configured date range.')

## EUV Download

Obtain dates to download.

In [ ]:
fill_between_he_dates = False

# Idenitfy He I observation datetimes
available_he_dates = [
    datetime.strptime(date_str, DICT_DATE_STR_FORMAT).date()
    for date_str in HE_DATE_LIST
]

if fill_between_he_dates:
    # Identify missing dates between He I observations
    days_in_period = (
        available_he_dates[-1] - available_he_dates[0]
    ).days
    all_period_dates = set(available_he_dates[0] + timedelta(num_days)
                        for num_days in range(days_in_period + 1))
    missing_he_dates = all_period_dates - set(available_he_dates)
    missing_he_date_str_list = [
        datetime.strftime(missing_date, DICT_DATE_STR_FORMAT)
        for missing_date in missing_he_dates
    ]
    # Set a place holder hour for missing dates
    missing_he_date_str_list = [
        missing_date_str[:12] + '16_00'
        for missing_date_str in missing_he_date_str_list
    ]
    missing_he_date_str_list.sort()

    all_period_date_str_list = HE_DATE_LIST + missing_he_date_str_list
    all_period_date_str_list.sort()
    
    all_period_date_list = list(all_period_dates)
    all_period_date_list.sort()
else:
    all_period_date_str_list = HE_DATE_LIST
    all_period_date_list = available_he_dates

# Identify datetimes to download EUV that have not yet been downloaded
available_euv_dates = [
    datetime.strptime(date_str, DICT_DATE_STR_FORMAT).date()
    for date_str in EUV_DATE_LIST
]
download_euv_date_list = [
    date_str for date_str, date
    in zip(all_period_date_str_list, all_period_date_list)
    if date not in available_euv_dates
]

In [ ]:
print('Datetimes to Download for EUV Observations:')
prepare_data.display_dates(download_euv_date_list)

Automatic

In [ ]:
prepare_data.download_euv(
    download_euv_date_list, EUV_DATE_LIST,
    sat='SDO',
    # sat='SOHO',
    output_dir=EUV_DIR, hr_window=2
)

Specify an Individual EUV-Observing Satellite and Datetime

In [ ]:
sat = 'SDO'
euv_date_str = '2015_06_11__04_00'
hr_window = 2

center_date = datetime.strptime(euv_date_str, DICT_DATE_STR_FORMAT)
        
min_date = center_date - timedelta(hours=hr_window)
max_date = center_date + timedelta(hours=hr_window)
time_range = a.Time(min_date, max_date)
cadence = a.Sample(30*u.minute)

if sat == 'SOHO':
    result = Fido.search(
        time_range,
        a.Instrument.eit, a.Wavelength(195*u.angstrom),
        cadence
    )
else: # sat == SDO
    result = Fido.search(
        time_range,
        a.Instrument.aia, a.Wavelength(193*u.angstrom),
        cadence
    )
result

In [ ]:
row_num = 0

Fido.fetch(result[:, row_num], path=EUV_DIR+'{file}')

## Rename Data

Be sure to re-run the **Load Data** section after renaming.

In [ ]:
# Delete all gzipped files after renaming
remove_gzip = True

# Rename all He, magnetogram, and EUV FITS files to include
# observation date in title
prepare_data.rename_dir(HE_DIR, remove_gzip)
prepare_data.rename_dir(MAG_DIR, remove_gzip)
prepare_data.rename_dir(EUV_DIR, remove_gzip)

# Available Data

In [ ]:
print(f'{len(HE_DATE_LIST)} Available Datetimes for He I Observations:')
prepare_data.display_dates(HE_DATE_LIST)

In [ ]:
print(f'{len(MAG_DATE_LIST)} Available Datetimes for Magnetograms:')
prepare_data.display_dates(MAG_DATE_LIST)

In [ ]:
print(f'{len(EUV_DATE_LIST)} Available Datetimes for EUV Observations:')
prepare_data.display_dates(EUV_DATE_LIST)

In [ ]:
print('He I Observation Coverage: '
      f'{len(HE_DATE_LIST)/len(EUV_DATE_LIST)*100:.1f}%')

In [ ]:
print('Available Datetimes for ACWE Confidence Maps:')
prepare_data.display_dates(ACWE_DATE_LIST)

Observation Comparison: He, Mag, EUV

In [ ]:
smooth_size_percent = 10

for he_date_str in HE_DATE_LIST:

    mag_date_str = prepare_data.get_nearest_date_str(
        MAG_DATE_LIST, selected_date_str=he_date_str
    )
    euv_date_str = prepare_data.get_nearest_date_str(
        date_str_list=EUV_DATE_LIST, selected_date_str=he_date_str
    )
    
    he_fits_file = DATA_FITS_FORMAT.format(
        data_dir=HE_DIR, date_str=he_date_str
    )
    mag_fits_file = DATA_FITS_FORMAT.format(
        data_dir=MAG_DIR, date_str=mag_date_str
    )
    euv_fits_file = DATA_FITS_FORMAT.format(
        data_dir=EUV_DIR, date_str=euv_date_str
    )
    he_map = prepare_data.get_nso_sunpy_map(he_fits_file)
    mag_map = prepare_data.get_nso_sunpy_map(mag_fits_file)
    euv_map = sunpy.map.Map(euv_fits_file)

    # Process magnetogram
    smoothed_mag_map = prepare_data.get_smoothed_map(mag_map, smooth_size_percent)

    fig = plt.figure(figsize=(18, 5))
    
    plot_detection.plot_he_map(fig, (1, 3, 1), he_map, he_date_str)

    ax = fig.add_subplot(132, projection=mag_map)
    mag_map.plot(axes=ax, vmin=-50, vmax=50, title=mag_date_str)
    plot_detection.plot_map_contours(ax, smoothed_mag_map)
    
    plot_detection.plot_euv_map(fig, (1, 3, 3), euv_map, euv_date_str)

# Data Products

Prepare and save pre-processed data and ensemble segmentation maps.

## Pre-Processed Files

### Pre-Processed Maps

In [ ]:
overwrite = False

if not os.path.isdir(PREPROCESS_MAP_SAVE_DIR):
    os.makedirs(PREPROCESS_MAP_SAVE_DIR)

for he_date_str in HE_DATE_LIST:
    
    # Optionally overwrite existing files
    pre_process_file = (PREPROCESS_MAP_SAVE_DIR + he_date_str
                        + '_pre_processed_map.fits')
    if os.path.isfile(pre_process_file) and not overwrite:
        print((f'He {he_date_str} pre-processed map already exists.'))
        continue
    
    he_fits_file = DATA_FITS_FORMAT.format(
        data_dir=HE_DIR, date_str=he_date_str
    )
    he_map = prepare_data.get_nso_sunpy_map(he_fits_file)
    
    pre_processed_map = detect.pre_process_v0_5_1(he_map)
    
    pre_processed_map.save(pre_process_file, overwrite=overwrite)
    print(f'{he_date_str} Pre-Processed Map Saved')

### Reprojected Magnetograms

In [ ]:
overwrite = False
smooth_size_percent = 10

if not os.path.isdir(ROTATED_MAG_SAVE_DIR):
    os.makedirs(ROTATED_MAG_SAVE_DIR)

for he_date_str in HE_DATE_LIST:

    mag_date_str = prepare_data.get_nearest_date_str(
        MAG_DATE_LIST, selected_date_str=he_date_str
    )
    
    fits_file_name = f'{ROTATED_MAG_SAVE_DIR}Mag{mag_date_str}_He{he_date_str}'
    reprojected_fits_file = f'{fits_file_name}.fits'
    reprojected_smooth_fits_file = f'{fits_file_name}_smooth.fits'
    
    # Optionally overwrite existing files
    if (os.path.isfile(reprojected_fits_file) or \
        os.path.isfile(reprojected_smooth_fits_file)) and not overwrite:
        print((f'{mag_date_str} magnetogram reprojected '
                + f'to {he_date_str} already exists.'))
        continue
    
    # Extract He I observation
    he_map = prepare_data.get_nso_sunpy_map(HE_DIR + he_date_str + '.fts')
    if not he_map:
        print(f'{he_date_str} He I observation extraction failed.')
        continue
    
    # Extract Magnetogram observation
    mag_map = prepare_data.get_nso_sunpy_map(MAG_DIR + mag_date_str + '.fts')

    # Process magnetogram
    reprojected_mag_map = prepare_data.diff_rotate(
        input_map=mag_map, target_map=he_map
    )
    
    smoothed_map = prepare_data.get_smoothed_map(mag_map, smooth_size_percent)
    reprojected_smooth_map = prepare_data.diff_rotate(
        input_map=smoothed_map, target_map=he_map
    )
    
    # Save to FITS files
    reprojected_mag_map.save(reprojected_fits_file, overwrite=overwrite)
    reprojected_smooth_map.save(f'{fits_file_name}_smooth.fits', overwrite=overwrite)
    print(f'{mag_date_str} magnetogram reprojected to {he_date_str} maps saved.')

## Ensemble Maps

v1.1

In [ ]:
LDA_FILE_NAME = 'v1_1_LDA_model.pkl'

with open(LDA_FILE_NAME, 'rb') as lda_file:
    lda = pickle.load(lda_file)

In [ ]:
overwrite = False

# v1.0 SOLIS Design
percent_of_peak_list = [70, 70, 80, 90]
morph_radius_list = [   15, 17, 13, 13] # Mm
probability_threshold = 0.4
# probability_threshold = 0


if not os.path.isdir(DETECTION_MAP_SAVE_DIR):
    os.makedirs(DETECTION_MAP_SAVE_DIR)

for he_date_str in HE_DATE_LIST:
    
    # Optionally overwrite existing files
    ensemble_file = f'{DETECTION_MAP_SAVE_DIR}{he_date_str}_ensemble_map.fits'
    if os.path.isfile(ensemble_file) and not overwrite:
        print((f'He {he_date_str} ensemble map already exists.'))
        continue
    
    # Extract He I map
    he_fits_file = DATA_FITS_FORMAT.format(
        data_dir=HE_DIR, date_str=he_date_str
    )
    he_map = prepare_data.get_nso_sunpy_map(he_fits_file)
    he_map_data = np.flipud(he_map.data)
    
    # Extract pre-processed map
    pre_process_file = (PREPROCESS_MAP_SAVE_DIR + he_date_str
                        + '_pre_processed_map.fits')
    pre_processed_map = sunpy.map.Map(pre_process_file)

    # Extract saved processed magnetograms
    mag_date_str = prepare_data.get_nearest_date_str(
        MAG_DATE_LIST, selected_date_str=he_date_str
    )
    reprojected_fits_file = (f'{ROTATED_MAG_SAVE_DIR}'
                             f'Mag{mag_date_str}_He{he_date_str}.fits')
    reprojected_mag_map = sunpy.map.Map(reprojected_fits_file)

    ensemble_map_data = detect.get_ensemble_v1_1(
        he_map_data, pre_processed_map, reprojected_mag_map,
        percent_of_peak_list, morph_radius_list, lda, probability_threshold
    )[0]
    ensemble_map = sunpy.map.Map(
        np.flipud(ensemble_map_data), pre_processed_map.meta
    )
    
    ensemble_map.save(ensemble_file, overwrite=overwrite)
    print(f'{he_date_str} Ensemble Map Saved')

v1.0

In [ ]:
overwrite = False

# v1.0 SOLIS Design
percent_of_peak_list = [70, 70, 80, 90]
morph_radius_list = [   15, 17, 13, 13] # Mm
unipolarity_threshold = 0.5
# unipolarity_threshold = 0

# # v1.0 KPVT Design
# percent_of_peak_list = [85, 105, 85, 95]
# morph_radius_list = [   17, 13, 15, 13] # Mm
# unipolarity_threshold = 0


if not os.path.isdir(DETECTION_MAP_SAVE_DIR):
    os.makedirs(DETECTION_MAP_SAVE_DIR)

for he_date_str in HE_DATE_LIST:
    
    # Optionally overwrite existing files
    ensemble_file = f'{DETECTION_MAP_SAVE_DIR}{he_date_str}_ensemble_map.fits'
    if os.path.isfile(ensemble_file) and not overwrite:
        print((f'He {he_date_str} ensemble map already exists.'))
        continue
    
    # Extract pre-processed map
    pre_process_file = (PREPROCESS_MAP_SAVE_DIR + he_date_str
                        + '_pre_processed_map.fits')
    pre_processed_map = sunpy.map.Map(pre_process_file)

    # Extract saved processed magnetograms
    mag_date_str = prepare_data.get_nearest_date_str(
        MAG_DATE_LIST, selected_date_str=he_date_str
    )
    reprojected_fits_file = (f'{ROTATED_MAG_SAVE_DIR}'
                             f'Mag{mag_date_str}_He{he_date_str}.fits')
    reprojected_mag_map = sunpy.map.Map(reprojected_fits_file)

    ensemble_map_data = detect.get_ensemble_v1_0(
        pre_processed_map, reprojected_mag_map,
        percent_of_peak_list, morph_radius_list,
        unipolarity_threshold
    )[0]
    ensemble_map = sunpy.map.Map(
        np.flipud(ensemble_map_data), pre_processed_map.meta
    )
    
    ensemble_map.save(ensemble_file, overwrite=overwrite)
    print(f'{he_date_str} Ensemble Map Saved')

## Fused Maps

In [ ]:
overwrite = False

if not os.path.isdir(FUSED_MAP_SAVE_DIR):
    os.makedirs(FUSED_MAP_SAVE_DIR)

for he_date_str in HE_DATE_LIST:
    
    # Optionally overwrite existing files
    fused_file = f'{FUSED_MAP_SAVE_DIR}{he_date_str}_fused_map.fits'
    if os.path.isfile(fused_file) and not overwrite:
        print((f'He {he_date_str} ensemble map already exists.'))
        continue
    
    # Extract saved ACWE map
    acwe_date_str = prepare_data.get_nearest_date_str(
        ACWE_DATE_LIST, he_date_str
    )
    acwe_map = prepare_data.get_acwe_sunpy_map(
        acwe_date_str, ACWE_DATE_LIST
    )
    
    # Extract saved ensemble map
    ensemble_file = f'{DETECTION_MAP_SAVE_DIR}{he_date_str}_ensemble_map.fits'
    ensemble_map = sunpy.map.Map(ensemble_file)
    ensemble_map_data = np.flipud(ensemble_map.data)

    # Align ACWE map to STRIDE map datetime via differential rotation
    rotated_acwe_map = prepare_data.diff_rotate(acwe_map, ensemble_map)
    rotated_acwe_map_data = np.flipud(rotated_acwe_map.data)

    # Fuse by taking pixel-wise maximum confidence
    fused_map_data = np.max(
        np.stack((ensemble_map_data, rotated_acwe_map_data)), axis=0
    )
    fused_map = sunpy.map.Map(np.flipud(fused_map_data), ensemble_map.meta)
    
    fused_map.save(fused_file, overwrite=overwrite)
    print(f'{he_date_str} Ensemble Map Saved')

# Data Product Plots

## Write Images to Video

Run directly after saving many images to a single output directory to compile them into a video.

In [ ]:
detect.write_video(out_dir, fps=2)

## Pre-Process Comparison

He I and pre-processed He I

In [ ]:
for he_date_str in HE_DATE_LIST:
    
    he_fits_file = DATA_FITS_FORMAT.format(
        data_dir=HE_DIR, date_str=he_date_str
    )
    he_map = prepare_data.get_nso_sunpy_map(he_fits_file)
    if not he_map:
        print(f'{he_date_str} He I observation extraction failed.')
        continue
    
    pre_process_file = (PREPROCESS_MAP_SAVE_DIR + he_date_str
                        + '_pre_processed_map.fits')
    pre_processed_map = sunpy.map.Map(pre_process_file)

    fig = plt.figure(figsize=(10, 4))
    
    plot_detection.plot_he_map(fig, (1, 2, 1), he_map, he_date_str)
    
    ax = fig.add_subplot(122, projection=pre_processed_map)
    pre_processed_map.plot(axes=ax, title=he_date_str)

## Ensemble Comparison

Plot ensemble segmentation maps alongside pre-processed data.

### EUV Comparison: He I, Ensemble with Inversion Lines, & EUV

v1.0+

In [ ]:
overwrite = True
out_dir = DETECTION_IMAGE_DIR + 'EUV_Comparison/'

if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

for euv_date_str in EUV_DATE_LIST:
    
    # Optionally overwrite existing files
    comparison_img_file = f'{out_dir}EUV{euv_date_str}.jpg'
    if os.path.isfile(comparison_img_file) and not overwrite:
        print((f'EUV {euv_date_str} comparison already exists.'))
        continue
    
    he_date_str = prepare_data.get_latest_date_str(
        HE_DATE_LIST, selected_date_str=euv_date_str,
        hr_window=7
        
    )
    mag_date_str = prepare_data.get_nearest_date_str(
        MAG_DATE_LIST, selected_date_str=he_date_str
    )
    
    fig = plt.figure(figsize=(21, 5))
    plot_detection.plot_he_neutral_lines_euv_v1_0(
        fig, he_date_str, mag_date_str, euv_date_str
    )
    
    # Save plot
    plt.savefig(comparison_img_file)
    plt.close(fig)
    print(f'{euv_date_str} map comparison saved.')

v0.5.1 & vY

In [ ]:
overwrite = True
# Option to display Heliographic reprojected maps. Leave as False
hg_reproject = False
out_dir = DETECTION_IMAGE_DIR + 'EUV_Comparison/'

if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

for euv_date_str in EUV_DATE_LIST:
    
    # Optionally overwrite existing files
    comparison_img_file = f'{out_dir}EUV{euv_date_str}.jpg'
    if os.path.isfile(comparison_img_file) and not overwrite:
        print((f'EUV {euv_date_str} comparison already exists.'))
        continue
    
    he_date_str = prepare_data.get_latest_date_str(
        HE_DATE_LIST, selected_date_str=euv_date_str
    )
    mag_date_str = prepare_data.get_nearest_date_str(
        MAG_DATE_LIST, selected_date_str=he_date_str
    )
    
    if hg_reproject:
        fig = plt.figure(figsize=(22, 5))
    else:
        fig = plt.figure(figsize=(18, 5))
    
    plot_detection.plot_he_neutral_lines_euv_v0_5_1(
        fig, he_date_str, mag_date_str, euv_date_str, hg_reproject=False
    )
    
    # Save plot
    plt.savefig(comparison_img_file)
    plt.close(fig)
    print(f'{euv_date_str} map comparison saved.')

v0.2-v0.5

In [ ]:
overwrite = False
out_dir = DETECTION_IMAGE_DIR + 'EUV_Comparison/'

if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

for euv_date_str in EUV_DATE_LIST:
    
    # Optionally overwrite existing files
    comparison_img_file = f'{out_dir}EUV{euv_date_str}.jpg'
    if os.path.isfile(comparison_img_file) and not overwrite:
        print((f'EUV {euv_date_str} comparison already exists.'))
        continue
    
    he_date_str = prepare_data.get_latest_date_str(
        HE_DATE_LIST, selected_date_str=euv_date_str
    )
    mag_date_str = prepare_data.get_nearest_date_str(
        MAG_DATE_LIST, selected_date_str=he_date_str
    )
    
    fig = plt.figure(figsize=(18, 5))
    plot_detection.plot_he_neutral_lines_euv_comparison(
        fig, he_date_str, mag_date_str, euv_date_str,
        ROTATED_MAG_SAVE_DIR
    )
    
    # Save plot
    plt.savefig(comparison_img_file)
    plt.close(fig)
    print(f'{euv_date_str} map comparison saved.')

Version Comparison

In [ ]:
overwrite = True
# out_dir = (DETECTION_IMAGE_DIR + '_Version_Comparison'
#            + DATE_DIR + 'v0_2_v0_4_Unipolar/')
out_dir = (DETECTION_IMAGE_DIR + '_Version_Comparison'
           + DATE_DIR + 'v0_5_v0_6/')

if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

# comparison_version_dir = DETECT_DIR + 'v0_1/' + 'Saved_npy_Files/'
comparison_version_dir = DETECT_DIR + 'v0_5/' + 'Saved_npy_Files/'

for euv_date_str in EUV_DATE_LIST:
    
    # Optionally overwrite existing files
    comparison_img_file = f'{out_dir}EUV{euv_date_str}.jpg'
    if os.path.isfile(comparison_img_file) and not overwrite:
        print((f'EUV {euv_date_str} comparison already exists.'))
        continue
    
    he_date_str = prepare_data.get_latest_date_str(
        HE_DATE_LIST, selected_date_str=euv_date_str
    )
    mag_date_str = prepare_data.get_nearest_date_str(
        MAG_DATE_LIST, selected_date_str=he_date_str
    )
    
    # Extract He I observation
    he_map = prepare_data.get_nso_sunpy_map(HE_DIR + he_date_str + '.fts')
    if not he_map:
        print(f'{he_date_str} He I observation extraction failed.')
    
    # Extract comparison saved ensemble map array and convert to Sunpy map
    comparison_file = f'{comparison_version_dir}{he_date_str}_ensemble_map.npy'
    comparison_map_data = np.load(comparison_file, allow_pickle=True)[-1]
    comparison_map_data = np.where(np.isnan(ensemble_map_data), np.nan, comparison_map_data)
    comparison_map = sunpy.map.Map(np.flipud(comparison_map_data), he_map.meta)
    
    # Extract saved ensemble map array and convert to Sunpy map
    ensemble_file = f'{DETECTION_SAVE_DIR}{he_date_str}_ensemble_map.npy'
    ensemble_map_data = np.load(ensemble_file, allow_pickle=True)[-1]
    ensemble_map = sunpy.map.Map(np.flipud(ensemble_map_data), he_map.meta)
    ensemble_map.plot_settings['cmap'] = colormaps['magma']

    
    # Extract saved processed magnetogram
    reprojected_smooth_file = (f'{ROTATED_MAG_SAVE_DIR}Mag{mag_date_str}'
                               + f'_He{he_date_str}_smooth.fits')
    reprojected_smooth_map = sunpy.map.Map(reprojected_smooth_file)
    
    euv_map = sunpy.map.Map(EUV_DIR + euv_date_str + '.fts')
    
    fig = plt.figure(figsize=(12, 11))
    
    # Plot He observation
    plot_detection.plot_he_map(fig, (2, 2, 1), he_map, he_date_str)
    
    plot_detection.plot_euv_map(fig, (2, 2, 2), euv_map, euv_date_str)
    
    # Plot ensemble map with overlayed neutral lines
    ax = fig.add_subplot(2, 2, 3, projection=he_map)
    comparison_map.plot(axes=ax, title='v0.5')
    plot_detection.plot_map_contours(ax, reprojected_smooth_map)
    
    # Plot ensemble map with overlayed neutral lines
    ax = fig.add_subplot(2, 2, 4, projection=he_map)
    ensemble_map.plot(axes=ax, title='v0.6')
    plot_detection.plot_map_contours(ax, reprojected_smooth_map)

    # Save plot
    plt.savefig(comparison_img_file)
    plt.close(fig)
    print(f'{euv_date_str} map comparison saved.')

### He I & Ensemble with Inversion Lines

In [ ]:
overwrite = True
out_dir = DETECTION_IMAGE_DIR + 'He_Comparison/'


if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

for he_date_str in HE_DATE_LIST[:1]:
    
    # Optionally overwrite existing files
    comparison_img_file = f'{out_dir}{he_date_str}.jpg'
    if os.path.isfile(comparison_img_file) and not overwrite:
        print((f'He I {he_date_str} comparison already exists.'))
        continue
    
    mag_date_str = prepare_data.get_nearest_date_str(
        MAG_DATE_LIST, selected_date_str=he_date_str
    )
    
    he_map = prepare_data.get_nso_sunpy_map(HE_DIR + he_date_str + '.fts')
    if not he_map:
        print(f'{he_date_str} He I observation extraction failed.')
    
    # Extract saved ensemble map array and convert to Sunpy map
    ensemble_file = f'{DETECTION_MAP_SAVE_DIR}{he_date_str}_ensemble_map.fits'
    ensemble_map = sunpy.map.Map(ensemble_file)
    
    # Extract saved processed magnetogram
    reprojected_smooth_file = (f'{ROTATED_MAG_SAVE_DIR}Mag{mag_date_str}'
                            + f'_He{he_date_str}_smooth.fits')
    reprojected_smooth_map = sunpy.map.Map(reprojected_smooth_file)

    fig = plt.figure(figsize=(12, 5))
    plot_detection.plot_he_map(fig, (1, 2, 1), he_map, he_date_str)
    
    # Plot ensemble map with overlayed neutral lines
    ax = fig.add_subplot(1, 2, 2, projection=ensemble_map)
    ensemble_map.plot(axes=ax, title='', cmap='magma')
    plot_detection.plot_map_contours(ax, reprojected_smooth_map)

    plt.savefig(comparison_img_file)
    plt.close(fig)
    print(f'{he_date_str} map saved.')

### Pre-Processed & Ensemble Maps

v1.0

In [ ]:
out_dir = DETECTION_IMAGE_DIR + 'Preprocess_Comparison/'


if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

for he_date_str in HE_DATE_LIST[:1]:
    
    pre_process_file = (PREPROCESS_MAP_SAVE_DIR + he_date_str
                        + '_pre_processed_map.fits')
    pre_processed_map = sunpy.map.Map(pre_process_file)
    
    ensemble_file = f'{DETECTION_MAP_SAVE_DIR}{he_date_str}_ensemble_map.fits'
    ensemble_map = sunpy.map.Map(ensemble_file)

    fig = plt.figure(figsize=(12, 5))
    ax = fig.add_subplot(1, 2, 1, projection=pre_processed_map)
    pre_processed_map.plot(axes=ax, title=he_date_str)
    
    ax = fig.add_subplot(1, 2, 2, projection=ensemble_map)
    ensemble_map.plot(axes=ax, title='', cmap='magma', vmin=0, vmax=1)

    # plt.savefig(f'{out_dir}{he_date_str}.jpg')
    # plt.close(fig)
    # print(f'{he_date_str} map saved.')

v0.5.1

In [ ]:
out_dir = DETECTION_IMAGE_DIR + 'Preprocess_Comparison/'


if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

for he_date_str in HE_DATE_LIST[:1]:
    
    pre_process_file = (PREPROCESS_MAP_SAVE_DIR + he_date_str
                        + '_pre_processed_map.fits')
    pre_processed_map = sunpy.map.Map(pre_process_file)
    
    ensemble_file = f'{DETECTION_MAP_SAVE_DIR}{he_date_str}_ensemble_map.fits'
    ensemble_map = sunpy.map.Map(ensemble_file)

    fig = plt.figure(figsize=(12, 5))
    ax = fig.add_subplot(1, 2, 1, projection=pre_processed_map)
    pre_processed_map.plot(axes=ax, title=he_date_str)
    
    ax = fig.add_subplot(1, 2, 2, projection=ensemble_map)
    ensemble_map.plot(axes=ax, title='', cmap='magma', vmin=0, vmax=100)

    # plt.savefig(f'{out_dir}{he_date_str}.jpg')
    # plt.close(fig)
    # print(f'{he_date_str} map saved.')

v0.2-v0.5

In [ ]:
out_dir = DETECTION_IMAGE_DIR + 'Preprocess_Comparison/'


if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

for he_date_str in HE_DATE_LIST:
    
    pre_process_file = (PREPROCESS_NPY_SAVE_DIR + he_date_str
                        + '_pre_processed_map.npy')
    pre_processed_map = np.load(pre_process_file, allow_pickle=True)[-1]
    
    ensemble_file = f'{DETECTION_SAVE_DIR}{he_date_str}_ensemble_map.npy'
    ensemble_map = np.load(ensemble_file, allow_pickle=True)[-1]

    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 10))
    ax = axes.ravel()

    ax[0].set_title(he_date_str, fontsize=24)
    ax[0].imshow(pre_processed_map, cmap=plt.cm.gray)
    
    ax[1].imshow(ensemble_map, cmap=plt.cm.magma)

    plt.savefig(f'{out_dir}{he_date_str}.jpg')
    plt.close(fig)
    print(f'{he_date_str} map saved.')

vY

In [ ]:
out_dir = DETECTION_IMAGE_DIR + 'Preprocess_Comparison/'


if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

for he_date_str in HE_DATE_LIST:
    
    pre_process_file = (PREPROCESS_MAP_SAVE_DIR + he_date_str
                        + '_pre_processed_map.fits')
    pre_processed_map = sunpy.map.Map(pre_process_file)
    
    ensemble_file = f'{DETECTION_MAP_SAVE_DIR}{he_date_str}_ensemble_map.fits'
    ensemble_map = sunpy.map.Map(ensemble_file)

    fig = plt.figure(figsize=(18, 5))
    ax = fig.add_subplot(1, 5, (1,2), projection=pre_processed_map)
    pre_processed_map.plot(axes=ax, title=he_date_str)
    
    # Plot ensemble map with overlayed neutral lines
    ax = fig.add_subplot(1, 5, (3,5), projection=ensemble_map)
    ensemble_map.plot(axes=ax, title='', cmap='magma')

    plt.savefig(f'{out_dir}{he_date_str}.jpg')
    plt.close(fig)
    print(f'{he_date_str} map saved.')

### Magnetic Comparison: He I, Ensemble, & Magnetogram with Inversion Lines

v0.2-v0.5 Only

In [ ]:
out_dir = DETECTION_IMAGE_DIR + 'Mag_Comparison/'

if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

for he_date_str in HE_DATE_LIST[-10:]:

    mag_date_str = prepare_data.get_nearest_date_str(
        MAG_DATE_LIST, selected_date_str=he_date_str
    )

    # Extract He I observation
    he_map = prepare_data.get_nso_sunpy_map(HE_DIR + he_date_str + '.fts')
    if not he_map:
        print(f'{he_date_str} He I observation extraction failed.')
        continue

    # Extract saved processed magnetograms
    mag_fits_name = f'{ROTATED_MAG_SAVE_DIR}Mag{mag_date_str}_He{he_date_str}'
    reprojected_fits_file = f'{mag_fits_name}.fits'
    reprojected_smooth_fits_file = f'{mag_fits_name}_smooth.fits'
    reprojected_mag_map = sunpy.map.Map(reprojected_fits_file)
    reprojected_smooth_map = sunpy.map.Map(reprojected_smooth_fits_file)

    # Extract saved ensemble map array and convert to Sunpy map
    ensemble_file = f'{DETECTION_SAVE_DIR}{he_date_str}_ensemble_map.npy'
    ensemble_map_data = np.load(ensemble_file, allow_pickle=True)[-1]
    ensemble_map = sunpy.map.Map(np.flipud(ensemble_map_data), he_map.meta)
    ensemble_map.plot_settings['cmap'] = colormaps['magma']

    fig = plt.figure(figsize=(18, 5))

    plot_detection.plot_he_map(fig, (1, 3, 1), he_map, he_date_str)

    ax = fig.add_subplot(132, projection=he_map)
    ensemble_map.plot(axes=ax, title='')
    plot_detection.plot_map_contours(ax, reprojected_smooth_map)

    ax = fig.add_subplot(133, projection=he_map)
    reprojected_mag_map.plot(axes=ax, vmin=-50, vmax=50, title=mag_date_str)
    plot_detection.plot_map_contours(ax, reprojected_smooth_map)

    plt.savefig(f'{out_dir}He{he_date_str}.jpg')
    plt.close(fig)

    print(f'{he_date_str} map comparison saved')

### Full Comparison: He I, Ensemble, Magnetogram, & EUV

v0.2-v0.5 Only

In [ ]:
overwrite = True
out_dir = DETECTION_IMAGE_DIR + 'Full_Comparison/'
smooth_size_percent = 10

if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

for euv_date_str in EUV_DATE_LIST:
    
    # Optionally overwrite existing files
    comparison_img_file = f'{out_dir}EUV{euv_date_str}.jpg'
    if os.path.isfile(comparison_img_file) and not overwrite:
        print((f'EUV {euv_date_str} full comparison already exists.'))
        continue

    he_date_str = prepare_data.get_latest_date_str(
        HE_DATE_LIST, selected_date_str=euv_date_str
    )
    mag_date_str = prepare_data.get_nearest_date_str(
        MAG_DATE_LIST, selected_date_str=he_date_str
    )
    
    # Extract He I observation
    he_map = prepare_data.get_nso_sunpy_map(HE_DIR + he_date_str + '.fts')
    if not he_map:
        print(f'{he_date_str} He I observation extraction failed.')
        continue
    
    mag_map = prepare_data.get_nso_sunpy_map(MAG_DIR + mag_date_str + '.fts')
    
    # Extract and crop EUV map to similar zoom level to other observations 
    euv_map = sunpy.map.Map(EUV_DIR + euv_date_str + '.fts')
    euv_map = euv_map.submap(
        bottom_left=SkyCoord(
            Tx=-1050*u.arcsec, Ty=-1050*u.arcsec,
                 frame=euv_map.coordinate_frame
        ),
        top_right=SkyCoord(
            Tx=1050*u.arcsec, Ty=1050*u.arcsec,
            frame=euv_map.coordinate_frame
        )
    )

    # Process magnetogram
    smoothed_mag_map = prepare_data.get_smoothed_map(mag_map, smooth_size_percent)
    
    # Extract saved ensemble map array and convert to Sunpy map
    ensemble_file = f'{DETECTION_SAVE_DIR}{he_date_str}_ensemble_map.npy'
    ensemble_map_data = np.load(ensemble_file, allow_pickle=True)[-1]
    ensemble_map = sunpy.map.Map(np.flipud(ensemble_map_data), he_map.meta)
    ensemble_map.plot_settings['cmap'] = colormaps['magma']
    
    fig = plt.figure(figsize=(24, 5))

    plot_detection.plot_he_map(fig, (1, 4, 1), he_map, he_date_str)

    ax = fig.add_subplot(142, projection=he_map)
    ensemble_map.plot(axes=ax, title='')
    
    ax = fig.add_subplot(143, projection=mag_map)
    mag_map.plot(axes=ax, vmin=-50, vmax=50, title=mag_date_str)
    plot_detection.plot_map_contours(ax, smoothed_mag_map)
    
    plot_detection.plot_euv_map(fig, (1, 4, 4), euv_map, euv_date_str)
    
    plt.savefig(comparison_img_file)
    plt.close(fig)

    print(f'{euv_date_str} map comparison saved.')

## Single Mask Comparison: Pre-Processed He I & Single Mask

v0.1 Only

In [ ]:
out_dir = DETECTION_IMAGE_DIR + 'Preprocess_Single_Comparison/'


if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

for he_date_str in HE_DATE_LIST:
    
    pre_process_file = (PREPROCESS_NPY_SAVE_DIR + he_date_str
                        + '_pre_processed_map.npy')
    pre_processed_map = np.load(pre_process_file, allow_pickle=True)[-1]
    
    he_fits_file = DATA_FITS_FORMAT.format(
        data_dir=HE_DIR, date_str=he_date_str
    )
    raw_he = prepare_data.get_image_from_fits(he_fits_file)
    he = np.where(raw_he == raw_he[0,0], np.NaN, raw_he)
    
    mask_file = f'{DETECTION_SAVE_DIR}{he_date_str}_ensemble_map.npy'
    single_mask = np.load(mask_file, allow_pickle=True)[-1]

    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 10))
    ax = axes.ravel()

    ax[0].set_title(he_date_str, fontsize=24)
    ax[0].imshow(pre_processed_map, cmap=plt.cm.gray)
    
    ax[1].imshow(he, cmap=plt.cm.afmhot, vmin=-100, vmax=100)
    ax[1].contour(single_mask, linewidths=2, cmap=plt.cm.gray)

    plt.savefig(f'{out_dir}{he_date_str}.jpg')
    plt.close(fig)
    print(f'{he_date_str} map saved.')

In [ ]:
for euv_date_str in EUV_DATE_LIST[:1]:

    he_date_str = prepare_data.get_latest_date_str(
        HE_DATE_LIST, selected_date_str=euv_date_str
    )
    
    # Extract He I observation
    he_map = prepare_data.get_nso_sunpy_map(HE_DIR + he_date_str + '.fts')
    if not he_map:
        print(f'{he_date_str} He I observation extraction failed.')
        continue
    
    he_base_data = np.where(he_map.data == he_map.data[0,0], np.nan, he_map.data)
    he_base_map = sunpy.map.Map(he_base_data, he_map.meta)
    
    # Extract saved single mask array and convert to Sunpy map
    mask_file = f'{DETECTION_SAVE_DIR}{he_date_str}_ensemble_map.npy'
    mask_data = np.load(mask_file, allow_pickle=True)[-1]
    mask_map = sunpy.map.Map(np.flipud(mask_data), he_map.meta)
    mask_map.plot_settings['cmap'] = colormaps['gray']
    
    euv_map = sunpy.map.Map(EUV_DIR + euv_date_str + '.fts')
    
    fig = plt.figure(figsize=(18, 5))
    
    plot_detection.plot_he_map(fig, (1, 3, 1), he_map, he_date_str)
    
    # Plot He I observation with overlayed detection contours
    ax = fig.add_subplot(132, projection=he_map)
    he_base_map.plot(axes=ax, vmin=-100, vmax=100, title=he_date_str,
                     cmap='afmhot')
    for contour in mask_map.contour(0):
        ax.plot_coord(contour, color='black', linewidth=1)
    
    plot_detection.plot_euv_map(fig, (1, 3, 3), euv_map, euv_date_str)